<a href="https://colab.research.google.com/github/phelipe-sb/Rain-Prediction/blob/main/Predictions_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask
from flask_restful import Resource, Api, reqparse
import os
import json
from google.cloud import bigquery
from google.oauth2 import service_account

client = bigquery.Client(project='weather-project-305419')
app = Flask(__name__)
api = Api(app)

class Weather_Prediction(Resource):
    def get(self):

      query = """ SELECT *
                  FROM (
                  SELECT prediction_timestamp
                      , city
                      , rain_probability
                      , ROW_NUMBER() OVER (PARTITION BY prediction_timestamp, city ORDER BY prediction_datetime DESC, city) AS row_num
                  FROM `weather-project-305419.Daily_Weather.Prediction_Results` AS weather_pred
                  ORDER BY prediction_timestamp DESC
                  )
                  WHERE row_num = 1
                  LIMIT 50
              """
      df = (
      client.query(query)
      .result()
      .to_dataframe()
      )
      predictions = json.dumps(df.values.tolist())
        # return our data and 200 OK HTTP code
        return {'predictions': predictions}, 200

api.add_resource(Weather_Prediction, '/predictions')

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=int(os.environ.get('PORT', 8080)))